In [40]:
import pandas as pd
import plotly.express as px
import math


items=pd.read_csv('./data/weekly_item.csv')
items['WeekStartDate']=pd.to_datetime (items['WeekStartDate'])
remove_ordertypes=['SimpleCater','EZ Cater (Pickup)','Catering (Pickup)', 'EZ Cater (Delivery)','Olo Catering (Self-Delivery))','Olo Catering (Pickup)']
items=items[~items['OrderType'].isin(remove_ordertypes)]
items.OrderType.unique()


# remove items starting with google order as they  are not clear, can't get info from their item id, same ids are used for differtn items
items1=items[~items['MenuItemName'].str.contains('Google Order')]
items1['MenuItemName'].nunique()

items1['OrderType'] =items1['OrderType'].replace( {
'Uber Eats - Delivery!':'UEats-Delivery',
'Online Ordering (Pickup) *': 'Online-Pickup',
'UberEats (Pickup)': 'UEats-Pickup',
'Online Ordering (Dispatch) *': 'Online-Dispatch',
'Telephone - Pickup':'Telephone-Pickup',
 'Dine In, Dine In':'Dine In',
 'Take Out, Take Out':'Take Out',
 })

items1['MenuItemName'].nunique()
items1['OrderType'].value_counts()


/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_95860/3880818882.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



OrderType
Dine In                     18841
Take Out                    17510
UEats-Delivery              15995
Online-Pickup               13062
UEats-Pickup                 6183
Online-Dispatch              5994
Telephone-Pickup             2845
Google Online Ordering       2831
TimeShark (Pickup)           1193
UberEats                       23
Postmates Ordering             20
Ritual Ordering                16
Grubhub (Courier)               6
Uber Eats - Takeout!            3
Google Online (Dispatch)        2
Name: count, dtype: int64

In [41]:

item_volume=items1.groupby(['StoreId',	'OrderType','WeekStartDate','MenuItemName']).sum().reset_index()


# # when we set thrshold >50 only ubereats filters=> the high irder volumn is from ubereats, so we use >0.85 to capture other orderlines
threshold = item_volume['item_weekly_volumn'].quantile(0.75)
# threshold=40
high_volume_items = item_volume[item_volume['item_weekly_volumn'] >= threshold]
print(high_volume_items['item_weekly_volumn'].describe())
high_volume_items.sort_values(by='item_weekly_volumn',ascending=False,inplace=True)
high_volume_items.MenuItemName.nunique()
high_volume_items=high_volume_items[high_volume_items['MenuItemName']!='IMMUNITY SHOT*']
# 4 weeks exective dclined 
import pandas as pd
def show_graph(filtered_df,weeks,months):
    fig = px.line(
        filtered_df,
        x='WeekStartDate',
        y='item_weekly_volumn',
        color='MenuItemName',  # Highlights decline segment vs normal

        facet_col='OrderType',
        facet_row='StoreId',
        markers=True,
        title=f'Consective {weeks}-Week Declining Volume Over Last {months}-Months'
    )

    # Clean subplot labels
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig.update_layout(
        annotations=[a for a in fig.layout.annotations if a.text not in ['StoreId', 'OrderType']],
        yaxis_title=None
    )
    fig.update_yaxes(matches=None)
    fig.show()

def x_weeks_consecutive_decline_y_month(df, weeks=4, months=3,show_graphs=True):
    """
    Flags each week's data for menu items that had `weeks` consecutive declines
    in the last `months` months.

    Returns original DataFrame with an added column 'DeclineFlag' = 'decline' or 'normal'
    """

    df = df.copy()
    df['WeekStartDate'] = pd.to_datetime(df['WeekStartDate'])

    # Time window: last Y months excluding the latest week
    max_week = df['WeekStartDate'].max()
    cutoff = max_week - pd.DateOffset(months=months)
    df_recent = df[
        (df['WeekStartDate'] >= cutoff) &
        (df['WeekStartDate'] < max_week)
    ].copy()

    # Sort for reliable comparison
    df_recent = df_recent.sort_values(['StoreId', 'OrderType', 'MenuItemName', 'WeekStartDate'])

    # Initialize flag column
    df_recent['DeclineFlag'] = 'normal'

    # Define logic to find and flag X consecutive declining weeks
    def flag_consecutive_declines(group):
        vols = group['item_weekly_volumn'].values
        flags = ['normal'] * len(vols)
        count = 0

        for i in range(1, len(vols)):
            if vols[i] < vols[i - 1]:
                count += 1
                if count >= weeks:
                    # Mark the last `weeks` rows as 'decline'
                    for j in range(i - weeks + 1, i + 1):
                        flags[j] = 'decline'
            else:
                count = 0

        group['DeclineFlag'] = flags
        return group

    # Apply per item
    flagged_df = df_recent.groupby(['StoreId', 'OrderType', 'MenuItemName'], group_keys=False).apply(flag_consecutive_declines)
    # Filter for rows where is_decline_segment == 'decline'
    decline_rows = flagged_df[flagged_df['DeclineFlag'] == 'decline']

    # Get unique combinations that have any decline
    decline_combinations = decline_rows[['StoreId', 'OrderType', 'MenuItemName']].drop_duplicates()

    # Now filter the original DataFrame to only include those combinations
    filtered_df = flagged_df.merge(decline_combinations, on=['StoreId', 'OrderType', 'MenuItemName'], how='inner')
    
    if show_graphs:
        show_graph(filtered_df,weeks,months)
    
    return filtered_df
weeks=4
months=3
show_graphs=True
filtered_df=x_weeks_consecutive_decline_y_month(high_volume_items, weeks, months,show_graphs)


/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_95860/1682272198.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



count    22431.000000
mean        14.001204
std         15.160598
min          6.000000
25%          7.000000
50%         10.000000
75%         15.000000
max        285.000000
Name: item_weekly_volumn, dtype: float64


/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_95860/1682272198.py:82: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [62]:
ccw=filtered_df[(filtered_df['StoreId']==963260) & (filtered_df['OrderType']=='UEats-Delivery') &(filtered_df['MenuItemName']=='CHICKEN CAESAR WRAP')].sort_values(by='WeekStartDate')[['StoreId','OrderType','WeekStartDate','MenuItemName','item_weekly_volumn','item_weekly_revenue']]
ccw               


,StoreId,OrderType,WeekStartDate,MenuItemName,item_weekly_volumn,item_weekly_revenue
32,963260,UEats-Delivery,2025-02-03,CHICKEN CAESAR WRAP,99,2134.01
33,963260,UEats-Delivery,2025-02-10,CHICKEN CAESAR WRAP,90,1939.10
34,963260,UEats-Delivery,2025-02-17,CHICKEN CAESAR WRAP,78,1623.22
35,963260,UEats-Delivery,2025-02-24,CHICKEN CAESAR WRAP,28,546.22
36,963260,UEats-Delivery,2025-03-03,CHICKEN CAESAR WRAP,27,534.73
37,963260,UEats-Delivery,2025-03-10,CHICKEN CAESAR WRAP,31,611.69
38,963260,UEats-Delivery,2025-03-17,CHICKEN CAESAR WRAP,23,468.27
39,963260,UEats-Delivery,2025-03-24,CHICKEN CAESAR WRAP,40,787.60
40,963260,UEats-Delivery,2025-03-31,CHICKEN CAESAR WRAP,25,533.25
41,963260,UEats-Delivery,2025-04-07,CHICKEN CAESAR WRAP,30,601.20


In [60]:
# Chicken Caesar Wrap 
# First 3 weeks: High period 
avg_high_period = ccw['item_weekly_volumn'].iloc[0:3].mean()

# Remaining 9 weeks: Low period
avg_low_period = ccw['item_weekly_volumn'].iloc[3:].mean()

pct_change = math.ceil(((avg_high_period-avg_low_period) / avg_high_period)*100)

print(f'Item volume decreased by {pct_change}% from the high period to the low period')


Item volume decreased by 68% from the high period to the low period


In [53]:
print(avg_high_period-avg_low_period)


60.44444444444444


In [44]:
liv_wrap=filtered_df[(filtered_df['StoreId']==963290) & (filtered_df['OrderType']=='UEats-Delivery') &(filtered_df['MenuItemName']=='LIV WRAP')].sort_values(by='WeekStartDate')[['WeekStartDate','item_weekly_volumn','OrderType']]
liv_wrap


,WeekStartDate,item_weekly_volumn,OrderType
122,2025-02-03,96,UEats-Delivery
123,2025-02-10,98,UEats-Delivery
124,2025-02-17,109,UEats-Delivery
125,2025-02-24,130,UEats-Delivery
126,2025-03-03,144,UEats-Delivery
127,2025-03-10,44,UEats-Delivery
128,2025-03-17,24,UEats-Delivery
129,2025-03-24,16,UEats-Delivery
130,2025-03-31,12,UEats-Delivery
131,2025-04-07,15,UEats-Delivery


In [57]:
#LIV WRAP
# First 5 weeks: High period
avg_high_period = liv_wrap['item_weekly_volumn'].iloc[0:5].mean()

# Remaining 7 weeks: Low period
avg_low_period = liv_wrap['item_weekly_volumn'].iloc[5:].mean()

pct_change = math.ceil(((avg_low_period- avg_high_period) / avg_high_period)*100)

print(f'Item volume decreased by {pct_change}% from the high period to the low period')


Item volume decreased by 83% from the high period to the low period


In [46]:
aux=items1[items1['MenuItemName'].isin(['LIV WRAP','CHICKEN CAESAR WRAP']) & (items1['OrderType']=='UEats-Delivery')].dropna()
fig = px.line(
    aux,
    x='WeekStartDate',
    y='item_weekly_volumn',
  # Highlights decline segment vs normal

    facet_col='MenuItemName',
    facet_row='StoreId',
    markers=True,
    title=f'Weekly Volume on Uber Eats - Delivery'
)

# Clean subplot labels
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(
    annotations=[a for a in fig.layout.annotations if a.text not in ['StoreId', 'OrderType','item_weekly_volumn']],
    yaxis_title=None
)
fig.update_yaxes(matches=None)
fig.show()
